In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import col


In [2]:
# Initialize Spark session with necessary package
spark = SparkSession.builder \
    .appName("WikipediaPageRank")\
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.12.0")\
    .getOrCreate()


:: loading settings :: url = jar:file:/Users/akanshamalviya/Documents/Python/anaconda3/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/akanshamalviya/.ivy2/cache
The jars for the packages stored in: /Users/akanshamalviya/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e7726645-7575-4985-921a-23e4e1452202;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.12.0 in central
	found commons-io#commons-io;2.8.0 in central
	found org.glassfish.jaxb#txw2;2.3.3 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.2.5 in central
:: resolution report :: resolve 377ms :: artifacts dl 35ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.12.0 from central in [default]
	commons-io#commons-io;2.8.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.2.5 from central in [default]
	org.glassfish.jaxb#txw2;2.3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       con

In [3]:
# Read XML file
file_path = "/Users/akanshamalviya/Downloads/simplewiki-latest-abstract.xml"
df = spark.read.format('com.databricks.spark.xml') \
    .options(rowTag='page') \
    .load(file_path)

In [15]:
#Extract relevant data from XML
links = df.select('Astronomy', ).rdd

ConnectionRefusedError: [Errno 61] Connection refused

In [4]:
# Step 3: Extract the required information
links = df.select('Astronomy', 'text').rdd

AttributeError: 'DataFrameReader' object has no attribute 'select'

In [ ]:
# Step 4: Implement the PageRank algorithm
lines = links.map(lambda row: (row.title, row.text))
links = lines.flatMap(lambda row: [(row[0], link) for link in row[1].split() if link.startswith('[[') and link.endswith(']]')])

ranks = links.groupByKey().mapValues(lambda x: 1.0)

for iteration in range(10):  # Run 10 iterations for convergence
    contribs = links.join(ranks).flatMap(
        lambda url_urls_rank: [(url_urls_rank[1][0], url_urls_rank[1][1] / len(list(url_urls_rank[1][0])))]
    )
    ranks = contribs.reduceByKey(lambda x, y: x + y).mapValues(lambda rank: rank * 0.85 + 0.15)

In [ ]:
# Step 5: Sort and print top ten PageRanks
top_ten_ranks = ranks.takeOrdered(10, key=lambda x: -x[1])
for (link, rank) in top_ten_ranks:
    print(f"{link}: {rank}")



In [ ]:
# Stop the Spark session
spark.stop()